# PyCity mSchools Analysis

## Analysis

## Code

In [258]:
import os
from pathlib import Path
import pandas as pd

## Loading the data and looking at properties

In [259]:
schools_path = Path('Resources/schools_complete.csv')
students_path = Path('Resources/students_complete.csv')

schools_df = pd.read_csv(schools_path)
students_df = pd.read_csv(students_path)

In [260]:
#Print columns, shape, and datatypes of dataframes 
for name, df in [('Schools',schools_df), ('Students',students_df)]:
    print('Dataframe for:', name)
    print('Columns:', df.columns)
    print('Shape:', df.shape)
    print('column dtypes:',df.dtypes)
    print('\n')


Dataframe for: Schools
Columns: Index(['School ID', 'school_name', 'type', 'size', 'budget'], dtype='object')
Shape: (15, 5)
column dtypes: School ID       int64
school_name    object
type           object
size            int64
budget          int64
dtype: object


Dataframe for: Students
Columns: Index(['Student ID', 'student_name', 'gender', 'grade', 'school_name',
       'reading_score', 'math_score'],
      dtype='object')
Shape: (39170, 7)
column dtypes: Student ID        int64
student_name     object
gender           object
grade            object
school_name      object
reading_score     int64
math_score        int64
dtype: object




In [261]:
# Count NA values in each column
for name, df in [('Schools',schools_df), ('Students',students_df)]:
    print(f'In dataframe {name}:')
    print(f'Column : count of NA values')
    for col in df.columns:
        print(f'{col} :', df[col].isna().sum())
    print('\n')

In dataframe Schools:
Column : count of NA values
School ID : 0
school_name : 0
type : 0
size : 0
budget : 0


In dataframe Students:
Column : count of NA values
Student ID : 0
student_name : 0
gender : 0
grade : 0
school_name : 0
reading_score : 0
math_score : 0




## Merge dataframes

In [262]:
# Ensure the School ID count matches the school name count in both datasets
# This is a check to ensure that there are no misspelled or other abbreviations used in the school name
# Assumption: School ID is a complete list of unique school identifiers and there are no incorrect entries
len(schools_df['School ID'].unique()) == len(schools_df['school_name'].unique()) == len(students_df['school_name'].unique())

True

In [263]:
merged_df = pd.merge(schools_df, students_df, on='school_name')
merged_df.shape

(39170, 11)

## Calculate District Summary Values

In [264]:
num_unqiue_schools = len(schools_df['School ID'].unique())
num_unqiue_schools

15

In [265]:
total_students = len(students_df['Student ID'].unique())
total_students

39170

In [266]:
total_budget = schools_df['budget'].sum()
total_budget

24649428

In [267]:
avg_math_score = merged_df['math_score'].mean()
avg_reading_score = merged_df['reading_score'].mean()
print(avg_math_score, avg_reading_score)

78.98537145774827 81.87784018381414


In [268]:
perc_pass_math = merged_df['math_score'].ge(70).sum() / total_students
perc_pass_math

0.749808526933878

In [269]:
perc_pass_read = merged_df['reading_score'].ge(70).sum() / total_students
perc_pass_read

0.8580546336482001

In [270]:
perc_pass_both = (merged_df['reading_score'].ge(70) & merged_df['math_score'].ge(70)).sum() / total_students
perc_pass_both

0.6517232575950983

In [296]:
district_summary = pd.DataFrame({
    'Number of unique schools' : [num_unqiue_schools],
    'Total Students' : f'{total_students:,}',
    'Total Budget' : f'${total_budget:,.2f}',
    'Avg Math Score' : f'{avg_math_score:.2f}%',
    'Avg Reading Score' : f'{avg_reading_score:.2f}%',
    'Percent Passing Math' : f'{perc_pass_math:.2%}',
    'Percent Passing Reading' : f'{perc_pass_read:.2%}',
    'Percent Passing Overall' : f'{perc_pass_both:.2%}'
})
district_summary

,Number of unique schools,Total Students,Total Budget,Avg Math Score,Avg Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Overall
0,15,"39,170","$24,649,428.00",78.99%,81.88%,74.98%,85.81%,65.17%


## Calculate School Summary Values

In [272]:
# Use the code provided to select the type per school from school_data
school_types = schools_df.set_index(["school_name"])["type"]

In [273]:
# Make a df copy so that I am not operating on the same memory location as district summary data
groupby_school = merged_df.copy().set_index(["school_name"]).groupby('school_name')

In [274]:
stu_per_school = groupby_school['Student ID'].count()

In [275]:
school_budgets = groupby_school['budget'].unique().transform(int)

In [276]:
per_stu_budget = (school_budgets / stu_per_school).transform(float)

In [277]:
school_avg_math = groupby_school['math_score'].mean()
school_avg_read = groupby_school['reading_score'].mean()

In [278]:
# take merged_df, filter for desired characteristic, groupby 'school_name', select a column ('Student ID'), and count all the values. divide this by students per school
school_pass_both_perc = merged_df[(merged_df['reading_score'].ge(70) & merged_df['math_score'].ge(70))].groupby('school_name')['Student ID'].count() / stu_per_school
school_pass_math_perc = merged_df[merged_df['math_score'].ge(70)].groupby('school_name')['Student ID'].count() / stu_per_school
school_pass_read_perc = merged_df[merged_df['reading_score'].ge(70)].groupby('school_name')['Student ID'].count() / stu_per_school


In [279]:
school_summary = pd.DataFrame({
    'School Type' : school_types,
    'Total Students' : stu_per_school,
    'Total School Budget' : school_budgets,
    'Per Student Budget' : per_stu_budget,
    'Average Math Score' : school_avg_math,
    'Average Reading Score' : school_avg_read,
    '% Passing Math' : school_pass_math_perc,
    '% Passing Reading' : school_pass_read_perc,
    '% Overall Passing' : school_pass_both_perc
})
per_school_summary = school_summary.copy()

In [280]:
# format summary values
per_school_summary['Total School Budget'] = school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary['Per Student Budget'] = school_summary['Per Student Budget'].map("${:.2f}".format)
per_school_summary['Average Math Score'] = school_summary['Average Math Score'].map('{:.2f}'.format)
per_school_summary['Average Reading Score'] = school_summary['Average Reading Score'].map('{:.2f}'.format)
per_school_summary['% Passing Math'] = school_summary['% Passing Math'].map('{:.2%}'.format)
per_school_summary['% Passing Reading'] = school_summary['% Passing Reading'].map('{:.2%}'.format)
per_school_summary['% Overall Passing'] = school_summary['% Overall Passing'].map('{:.2%}'.format)
per_school_summary


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,54.29%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51%,96.25%,89.23%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%


## Metrics!

## Highest performing Schools
(by % Overall Passing)

In [281]:
top_schools = school_summary.sort_values('% Overall Passing', ascending=False)
top_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,0.941335,0.970398,0.913348
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,0.932722,0.973089,0.909480
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,0.933924,0.971390,0.905995
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,0.938677,0.965396,0.905826
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,0.945946,0.959459,0.905405


## Lowest Performing Schools
(by % Overall Passing)

In [282]:
bottom_schools = school_summary.sort_values('% Overall Passing', ascending=False)
bottom_schools.tail(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,0.660576,0.812224,0.535392
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,0.667530,0.808630,0.535275
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,0.656839,0.813164,0.535139
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,0.659885,0.807392,0.532045
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,0.663666,0.802201,0.529882


## Scores by Grade

### Math Scores by grade

In [283]:
# Use the code provided to separate the data by grade
ninth_graders = merged_df[(merged_df["grade"] == "9th")]
tenth_graders = merged_df[(merged_df["grade"] == "10th")]
eleventh_graders = merged_df[(merged_df["grade"] == "11th")]
twelfth_graders = merged_df[(merged_df["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_grade_math_scores = ninth_graders.groupby('school_name')['math_score'].mean()
tenth_grader_math_scores = tenth_graders.groupby('school_name')['math_score'].mean()
eleventh_grader_math_scores = eleventh_graders.groupby('school_name')['math_score'].mean()
twelfth_grader_math_scores = twelfth_graders.groupby('school_name')['math_score'].mean()

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.DataFrame({
    "9th" : ninth_grade_math_scores,
    "10th" : tenth_grader_math_scores,
    "11th" : eleventh_grader_math_scores,
    "12th" : twelfth_grader_math_scores
})

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade


,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


### Reading Scoes by Grade

In [284]:
# Use the code provided to separate the data by grade
ninth_graders = merged_df[(merged_df["grade"] == "9th")]
tenth_graders = merged_df[(merged_df["grade"] == "10th")]
eleventh_graders = merged_df[(merged_df["grade"] == "11th")]
twelfth_graders = merged_df[(merged_df["grade"] == "12th")]

# Group by `school_name` and take the mean of the the `reading_score` column for each.
ninth_grade_reading_scores = ninth_graders.groupby('school_name')['reading_score'].mean()
tenth_grader_reading_scores = tenth_graders.groupby('school_name')['reading_score'].mean()
eleventh_grader_reading_scores = eleventh_graders.groupby('school_name')['reading_score'].mean()
twelfth_grader_reading_scores = twelfth_graders.groupby('school_name')['reading_score'].mean()

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade =pd.DataFrame({
    "9th" : ninth_grade_reading_scores,
    "10th" : tenth_grader_reading_scores,
    "11th" : eleventh_grader_reading_scores,
    "12th" : twelfth_grader_reading_scores
})

# Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by Spending

In [285]:
# Establish the bins
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [286]:
# Create a copy of the school summary since it has the "Per Student Budget"
school_spending_df = school_summary.copy()


In [287]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df['Per Student Budget'], spending_bins, right=False, labels=labels)
school_spending_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
school_name,,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,0.666801,0.819333,0.546423,$585-630
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,0.941335,0.970398,0.913348,<$585
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,0.659885,0.807392,0.532045,$630-645
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,0.683096,0.792990,0.542899,$630-645
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,0.933924,0.971390,0.905995,$585-630
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,0.667530,0.808630,0.535275,$645-680
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,0.925059,0.962529,0.892272,<$585
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,0.656839,0.813164,0.535139,$645-680
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,0.660576,0.812224,0.535392,$645-680


In [288]:
#  Calculate averages for the desired columns.
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()


In [289]:
spending_summary = pd.DataFrame({
    'Average math score' : spending_math_scores,
    'Average reading score' : spending_reading_scores,
    'Percent passing math'  : spending_passing_math * 100,
    'Percent passing reading'  : spending_passing_reading * 100,
    '% overall passing' : overall_passing_spending * 100
})
spending_summary

,Average math score,Average reading score,Percent passing math,Percent passing reading,% overall passing
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


## Scores by School Size

In [290]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [291]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.
per_school_summary = school_summary.copy()
per_school_summary["School Size"] = pd.cut(per_school_summary['Total Students'], bins=size_bins, labels=labels, right=False, include_lowest=True)
per_school_summary


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,School Size
school_name,,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,0.666801,0.819333,0.546423,Large (2000-5000)
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,0.941335,0.970398,0.913348,Medium (1000-2000)
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,0.659885,0.807392,0.532045,Large (2000-5000)
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,0.683096,0.792990,0.542899,Large (2000-5000)
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,0.933924,0.971390,0.905995,Medium (1000-2000)
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,0.667530,0.808630,0.535275,Large (2000-5000)
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,0.925059,0.962529,0.892272,Small (<1000)
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,0.656839,0.813164,0.535139,Large (2000-5000)
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,0.660576,0.812224,0.535392,Large (2000-5000)


In [292]:
# Calculate averages for the desired columns.
size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"])["% Overall Passing"].mean()


In [293]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.DataFrame([size_math_scores, 
    size_reading_scores, 
    size_passing_math * 100, 
    size_passing_reading * 100,
    size_overall_passing * 100
    ]).transpose()

# Display results
size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


In [294]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()


In [295]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame([
    average_math_score_by_type, 
    average_reading_score_by_type, 
    average_percent_passing_math_by_type * 100, 
    average_percent_passing_reading_by_type * 100, 
    average_percent_overall_passing_by_type * 100
    ]).transpose()

# Display results
type_summary


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
